In [4]:
#import the tools 
import pandas as pd
import numpy as np

# for inconsistent entry cleaning
import re
import fuzzywuzzy
from fuzzywuzzy import process

In [5]:
#import the dataset
df = pd.read_csv(r"C:\Users\Codie\Documents\Datasets\Coffee Data\Dirty data\coffee_fix.csv")

In [6]:
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,50/73,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,54/77,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,54/70,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,53/79,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,48/70,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [7]:
df.dtypes

slug            object
all_text        object
rating           int64
roaster         object
name            object
location        object
origin          object
roast           object
est_price       object
review_date     object
agtron          object
aroma          float64
acid           float64
body           float64
flavor         float64
aftertaste     float64
with_milk      float64
desc_1          object
desc_2          object
desc_3          object
dtype: object

In [8]:
#Checking for NaN values

df.isna().sum()

slug              0
all_text          0
rating            0
roaster           0
name              0
location          1
origin            0
roast            53
est_price         5
review_date       0
agtron            0
aroma            27
acid            335
body              3
flavor            3
aftertaste        3
with_milk      1926
desc_1            0
desc_2            0
desc_3            2
dtype: int64

In [9]:
#Dropping unneeded columns for analysis

df = df.drop(["slug", "all_text", "agtron","aroma", "acid", "body", "flavor", "aftertaste", "with_milk"], axis=1)

In [10]:
df.head()

,rating,roaster,name,location,origin,roast,est_price,review_date,desc_1,desc_2,desc_3
0,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [11]:
#Lets look at the ammount of data we have left

print(f"Length of the dataframe: {len(df)}")
print(" ")
print(f"Number of coffee brands: {len(df.name.unique())}")
print(" ")
print(f"Number of unique locations: {len(df.location.unique())}")


Length of the dataframe: 2282
 
Number of coffee brands: 2077
 
Number of unique locations: 296


In [12]:
df["location"].value_counts()

Madison, Wisconsin          162
Chia-Yi, Taiwan             146
Minneapolis, Minnesota      116
Taipei, Taiwan              111
San Diego, California       106
                           ... 
Shakopee, Minnesota           1
Matthews, North Carolina      1
Landover, Maryland            1
Seattle, Washingto            1
Taipai, Taiwan                1
Name: location, Length: 295, dtype: int64

In [13]:
df["origin"].value_counts()

Yirgacheffe growing region, southern Ethiopia    90
Guji Zone, Oromia Region, southern Ethiopia      61
Nyeri growing region, south-central Kenya        61
Not disclosed                                    45
Boquete growing region, western Panama           41
                                                 ..
Taiwan                                            1
Doi Pangkhon, Thailand                            1
Buenos Aires, Comayagua, Honduras                 1
Ka’u growing region, Big island of Hawai’i        1
Tarrazu, Costa RIca                               1
Name: origin, Length: 993, dtype: int64

In [14]:
# Both of these columns are a mess. We have to clean these columns to be more readable better for analysis

#split the strings by , \ and ; symbols
split_location = df["location"].str.split(",|;", expand=True)

#separate country and region
df["loc_country"] = split_location[1]
df["loc_region"] = split_location[0]

# do the same for the origin column
split_origin = df["origin"].str.split(",|;", expand=True)
df["origin_1"] = split_origin[0]
df["origin_2"] = split_origin[1]

In [15]:
#Lets get rid of the orignal columns

df.drop(["location", "origin"], axis=1)

,rating,roaster,name,roast,est_price,review_date,desc_1,desc_2,desc_3,loc_country,loc_region,origin_1,origin_2
0,95,A.R.C.,“Sweety” Espresso Blend,Medium-Light,HKD $250/227 grams,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,China,Hong Kong,Panama,Ethiopia
1,94,A.R.C.,Flora Blend Espresso,Medium-Light,HKD $158/227 grams,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...",China,Hong Kong,Africa,Asia Pacific
2,92,Revel Coffee,Ethiopia Shakiso Mormora,Medium-Light,$16.00/12 ounces,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...",Montana,Billings,Guji Zone,southern Ethiopia
3,92,Roast House,Ethiopia Suke Quto,Medium-Light,$19.00/16 ounces,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,Washington,Spokane,Guji Zone,Oromia Region
4,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,Medium,$16.50/12 ounces,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,Montana,Hamilton,Gedeb District,Gedeo Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,92,El Gran Cafe,Finca Patzibir,Medium-Light,$16.00/12 ounces,November 2022,"Crisply sweet, nut-toned. Almond brittle, pie ...",Produced by Andres Fahsen of Finca Patzibar en...,"A quietly confident, sweetly nut-toned Guatema...",Guatemala,Antigua,Lake Atitlán growing region,Guatemala
2278,93,El Gran Cafe,Proyecto Aurora,Medium-Light,$10.00/12 ounces,November 2022,"Chocolaty, floral-framed. Cocoa nib, honeysuck...",Produced at Beneficio La Esperanza from trees ...,"A deeply floral, richly chocolaty Guatemala cu...",Guatemala,Antigua,Antigua growing region,Guatemala
2279,93,El Gran Cafe,Finca El Potrero,Medium-Light,$10.00/12 ounces,November 2022,"High-toned, enticingly sweet. Black cherry, na...",Produced by Josué Morales of Finca El Potrero...,"A bright, balanced, juicy Guatemala cup driven...",Guatemala,Antigua,Antigua growing region,Guatemala
2280,93,El Gran Cafe,Chacayá Santiago Atitlán,Medium-Light,$16.00/12 ounces,November 2022,"Vibrantly sweet, subtly nuanced. Apricot, dar...",Produced by smallholding farmers of Asociación...,"Balanced, bright, invigoratingly crisp, with t...",Guatemala,Antigua,Lake Atitlán growing region,Guatemala


In [16]:
#Let's see how many unique regions we have now

regions = df["loc_region"] 
print(f"Unique regions: {len(regions.unique())}")

Unique regions: 271


In [17]:
countries = df["loc_country"] 

print(countries.unique())
print(f"Unique countries: {len(countries.unique())}")

[' China' ' Montana' ' Washington' ' Virginia' ' Connecticut'
 ' California' ' Wisconsin' ' Ontario' ' Kansas' ' Taiwan'
 ' British Columbia' nan ' Minnesota' ' Florida' ' Hawaii' ' Wyoming'
 ' Oregon' ' Colorado' ' Maine' ' Kentucky' ' New Hampshire' ' New Jersey'
 ' Sydney' ' Massachusetts' ' North Carolina' ' Thailand' ' Nova Scotia'
 ' Illinois' ' South Korea' ' Texas' ' Pennsylvania' ' Canada' ' New York'
 ' England' None ' Japan' ' United Arab Emirates' ' New Mexico'
 ' Californiaa' ' Idaho' ' Tennessee' ' Ohio' ' Uganda' ' Michigan'
 ' Oklahoma' ' MInnesota' ' Maui' ' Vermont' ' Chia-Yi' ' Georgia'
 " Hawai'i" ' Louisiana' ' Mississippi' ' Alabama' ' Hong Kong'
 ' Slovakia' ' Kauia' ' Oaxaca' ' Indonesia' ' Maryland' ' Laos'
 ' Washingto' ' Belgium' ' Nevada' ' Virginia and Floyd' ' Mexico' ' Iowa'
 ' D.C.' ' Alberta' ' Nantou' ' Missouri' ' Hawai’i' ' Guatemala'
 ' Satipo Province' ' Kauai' ' Honduras' ' Saskatchewan' ' Colombia'
 ' Quebec' ' Oregon ' ' Chia-yi' ' Alaska' ' Ken

In [18]:
# Note: Many of these "countries" are US/Mexican states or Canadian Provinces
# There are also some misspellings and typos
# removing spacing and Capitalizing each new word should remove some duplicates

def fix_text(df, col):
    # lowercase all letters
    df[col] = df[col].str.lower()
    #remove spaces at teh beginning and at the end of the string
    df[col] = df[col].str.strip()
    # capitalize each word
    df[col] = df[col].str.title()

In [19]:
fix_text(df, "loc_country")

print(f"Number of unique countries after fixing the text: {countries.nunique()}")

Number of unique countries after fixing the text: 90


In [20]:
# Let's give the same treatment to regions

fix_text(df,"loc_region")
print(f"Number of unique countries after fixing the text: {regions.nunique()}")

Number of unique countries after fixing the text: 270


In [21]:
matches = fuzzywuzzy.process.extract("Taipei", 
                                     countries, 
                                     limit=10, 
                                     scorer=fuzzywuzzy.fuzz.token_sort_ratio)

matches

[(' Taipei City', 71, 2077),
 (' New Taipei City', 57, 2219),
 (' Maine', 55, 46),
 (' Maine', 55, 77),
 (' Maine', 55, 299),
 (' Maine', 55, 419),
 (' Maine', 55, 457),
 (' Maine', 55, 1363),
 (' Maine', 55, 1401),
 (' Maine', 55, 1632)]

In [22]:
# It looks like it was mostly correct. But we always verify because Maine? Not even close
# Let's creat a function that will help us find matches and fix some countries that were misspelled
def replace_matches_in_column(df, column, string_to_match, min_ratio = 55):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match

In [23]:
titles_to_replace = ["Hawai'i", "California", "Washington"]

for i in titles_to_replace:
    replace_matches_in_column(df, "loc_country", i)

In [24]:
# Let's see how many unique contries we have now
print("Unique countries:", countries.nunique())

Unique countries: 90


In [25]:
regions_to_replace = ["Taipei", "New Taipei City", "Sacramento", "Chia-Yi", "Tainan", "Nantou"]

for i in regions_to_replace:
    replace_matches_in_column(df, "loc_region", i)

In [26]:
regions.nunique()

270

In [27]:
# Let's change States and cities to the country they belong to
df.loc[df["loc_region"] == "Hong Kong",["loc_country"]] = "Hong Kong"

In [28]:
to_replace = {"United States":["Montana", "Pennsylvania","United States And Floyd", "Washington", "Virginia", "Connecticut", "California", "Wisconsin", "Kansas", "Minnesota", "Florida", "Wyoming", "Oregon", "Colorado", "Maine", "Kentucky", "New Hampshire", "New Jersey", "Massachusetts", "North Carolina","Illinois", "Texas", "Pennsylvania", "New York", "New Mexico", "Idaho", "Tennessee", "Ohio", "Michigan", "Oklahoma", "Vermont", "Georgia", "Louisiana", "Mississippi", "Alabama", "Maryland", "Nevada", "Virginia and Floyd", "Iowa", "D.C.", "Missouri", "Oregon", "Alaska", "Arizona", "California", "Virginia", "Colorado", "United States And Floyd"],
             "Hawai'i":["Kauai", "Hawai'i", "Maui", "Kauia"],
            "Mexico":["Oaxaca", "Mexico"],
            "Canada":["Ontario", "Nova Scotia", "Quebec", "Saskatchewan", "British Columbia", "Alberta"],
            "Taiwan":["Taiwan", "Taipei City", "Taichung", "New Taipei City", "Nantou", "New Taiwan"],
            "Australia":["Sydney"],
             "Peru":["Satipo Province"],
             "Malaysia":["Sabah"]}

def replace_values(dictionary, df_column):
    for key, value in dictionary.items():
        df_column.replace(value, key, regex=True, inplace=True)

In [29]:
replace_values(to_replace, df.loc_country)

df.loc_country.unique()

array(['Hong Kong', 'United States', 'Canada', 'Taiwan', nan, "Hawai'i",
       'Australia', 'Thailand', 'South Korea', 'England', 'China', None,
       'Japan', 'United Arab Emirates', 'Uganda', 'Slovakia', 'Mexico',
       'Indonesia', 'Laos', 'Belgium', 'United States And Floyd',
       'Guatemala', 'Peru', 'Honduras', 'Kenya', 'Malaysia', 'New Taiwan'],
      dtype=object)

In [30]:
df.loc[df.loc_country.isna(), ["loc_country", "loc_region"]]

,loc_country,loc_region
15,NaN,Tainan
292,None,South Korea
713,None,Los Angeles
1748,None,Branford Connecticut
1964,None,Tainan


In [31]:
#Not to many missing. Let's just fill the in manually

df.at[15, "loc_country"] = "Taiwan"
df.at[292, "loc_country"] = "South Korea"
df.at[713, "loc_country"] = "United States"
df.at[1748, "loc_country"] = "United States"
df.at[1964, "loc_country"] = "Taiwan"

In [32]:
df.loc[df.loc_country.isna(), ["loc_country", "loc_region"]]

,loc_country,loc_region


In [33]:
#We can now drop loc_region
df = df.drop("loc_region", axis=1)

In [34]:
#Now we have to look at the two origin columns

df.loc[:,["origin_1", "origin_2"]]

,origin_1,origin_2
0,Panama,Ethiopia
1,Africa,Asia Pacific
2,Guji Zone,southern Ethiopia
3,Guji Zone,Oromia Region
4,Gedeb District,Gedeo Zone
...,...,...
2277,Lake Atitlán growing region,Guatemala
2278,Antigua growing region,Guatemala
2279,Antigua growing region,Guatemala
2280,Lake Atitlán growing region,Guatemala


In [35]:
df["origin_1"].value_counts()

Guji Zone                     147
Yirgacheffe growing region    115
Ethiopia                       89
Nyeri growing region           63
Boquete growing region         48
                             ... 
Llano Bonito                    1
La Providencia                  1
Djimma                          1
Santander                       1
Volcano                         1
Name: origin_1, Length: 644, dtype: int64

In [36]:
cols = ["origin_1", "origin_2"]

for i in cols:
    fix_text(df, i)

In [37]:
df["origin_2"].fillna(df["origin_1"].shift(), inplace=True)

In [38]:
df = df.drop(["location", "origin"], axis=1)

In [39]:
df.head()

,rating,roaster,name,roast,est_price,review_date,desc_1,desc_2,desc_3,loc_country,origin_1,origin_2
0,95,A.R.C.,“Sweety” Espresso Blend,Medium-Light,HKD $250/227 grams,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,Hong Kong,Panama,Ethiopia
1,94,A.R.C.,Flora Blend Espresso,Medium-Light,HKD $158/227 grams,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...",Hong Kong,Africa,Asia Pacific
2,92,Revel Coffee,Ethiopia Shakiso Mormora,Medium-Light,$16.00/12 ounces,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...",United States,Guji Zone,Southern Ethiopia
3,92,Roast House,Ethiopia Suke Quto,Medium-Light,$19.00/16 ounces,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,United States,Guji Zone,Oromia Region
4,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,Medium,$16.50/12 ounces,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,United States,Gedeb District,Gedeo Zone


In [40]:
#We now need to standardize currency
df["est_price"][:20]

0       HKD $250/227 grams
1       HKD $158/227 grams
2         $16.00/12 ounces
3         $19.00/16 ounces
4         $16.50/12 ounces
5         $17.49/12 ounces
6         $17.99/16 ounces
7         $17.50/12 ounces
8         $18.00/12 ounces
9     CAD $17.00/12 ounces
10        $24.50/12 ounces
11       NT $550/16 ounces
12    CAD $17.00/16 ounces
13        $19.50/12 ounces
14      HKD $128/227 grams
15      HKD $140/454 grams
16      HKD $130/454 grams
17      HKD $120/227 grams
18      HKD $128/454 grams
19      HKD $128/227 grams
Name: est_price, dtype: object

In [42]:
# That is a ton of different currency, measurements and even just weights! 
#Lets start simple, checking for missing values
df["est_price"].isna().sum()

5

In [43]:
df = df.dropna(subset=["est_price"])

In [44]:
# weight measurement
df.loc[df["est_price"].str.contains("grams", case=False), "grams"] = "grams"
df.loc[df["est_price"].str.contains("ounces", case=False), "ounces"] = "ounces"

# currency
df.loc[df["est_price"].str.contains("HKD", case=False), "HKD"] = "HKD"
df.loc[df["est_price"].str.contains("CAD", case=False), "CAD"] = "CAD"
df.loc[df["est_price"].str.contains("NT", case=False), "NT"] = "NT"
df.loc[df["est_price"].str.contains("¥"), "JPY"] = "JPY"
df.loc[df["est_price"].str.contains("€"), "EUR"] = "EUR"
df.loc[df["est_price"].str.contains("£"), "GBP"] = "GBP"

In [45]:
df.loc[:, ["est_price", "grams", "ounces", "HKD", "CAD", "NT", "JPY"]]

,est_price,grams,ounces,HKD,CAD,NT,JPY
0,HKD $250/227 grams,grams,NaN,HKD,NaN,NaN,NaN
1,HKD $158/227 grams,grams,NaN,HKD,NaN,NaN,NaN
2,$16.00/12 ounces,NaN,ounces,NaN,NaN,NaN,NaN
3,$19.00/16 ounces,NaN,ounces,NaN,NaN,NaN,NaN
4,$16.50/12 ounces,NaN,ounces,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2277,$16.00/12 ounces,NaN,ounces,NaN,NaN,NaN,NaN
2278,$10.00/12 ounces,NaN,ounces,NaN,NaN,NaN,NaN
2279,$10.00/12 ounces,NaN,ounces,NaN,NaN,NaN,NaN
2280,$16.00/12 ounces,NaN,ounces,NaN,NaN,NaN,NaN


In [46]:
# Let's start with HKD
HKD_prices = df.loc[df["HKD"].notna() == True, ["est_price"]]

HKD_prices

,est_price
0,HKD $250/227 grams
1,HKD $158/227 grams
14,HKD $128/227 grams
15,HKD $140/454 grams
16,HKD $130/454 grams
17,HKD $120/227 grams
18,HKD $128/454 grams
19,HKD $128/227 grams
20,HKD $120/227 grams
21,HKD $200/227 grams


In [47]:
#Get just the numbers

HKD_prices = HKD_prices.replace(r"[^\d/]+","", regex=True)
HKD_prices

,est_price
0,250/227
1,158/227
14,128/227
15,140/454
16,130/454
17,120/227
18,128/454
19,128/227
20,120/227
21,200/227


In [48]:
#Split the numbers
split_HKD = HKD_prices["est_price"].str.split("/", expand=True)
HKD_prices["HKD"] = split_HKD[0].astype("int")
HKD_prices["grams"] = split_HKD[1].astype("int")
HKD_prices

,est_price,HKD,grams
0,250/227,250,227
1,158/227,158,227
14,128/227,128,227
15,140/454,140,454
16,130/454,130,454
17,120/227,120,227
18,128/454,128,454
19,128/227,128,227
20,120/227,120,227
21,200/227,200,227


In [49]:
#Droping the column we don't need. And then convert.
HKD_prices.drop("est_price", axis=1, inplace=True)

HKD_prices["price_per_gram"] = HKD_prices["HKD"] / HKD_prices["grams"]*0.13
HKD_prices

,HKD,grams,price_per_gram
0,250,227,0.143172
1,158,227,0.090485
14,128,227,0.073304
15,140,454,0.040088
16,130,454,0.037225
17,120,227,0.068722
18,128,454,0.036652
19,128,227,0.073304
20,120,227,0.068722
21,200,227,0.114537


In [50]:
HKD_prices["100g_USD"] = round(HKD_prices["price_per_gram"] * 100, 2)
HKD_prices.drop(["HKD", "grams", "price_per_gram"], axis=1, inplace=True)
HKD_prices

,100g_USD
0,14.32
1,9.05
14,7.33
15,4.01
16,3.72
17,6.87
18,3.67
19,7.33
20,6.87
21,11.45


In [51]:
#Continue this process with all the other currencies
CAD_prices = df.loc[df["CAD"].notna() == True, ["est_price"]]
CAD_prices

,est_price
9,CAD $17.00/12 ounces
12,CAD $17.00/16 ounces
29,CAD $18.00/12 ounces
30,CAD $18.00/12 ounces
31,CAD $21.00/12 ounces
128,CAD $15.50/340 grams
163,CAD $29.00/200 grams
212,CAD $16.00/16 ounces
217,CAD $17.00/12 ounces
231,CAD $15.00/250 grams


In [52]:
def clean_df(df, currency, weight):
    #remove the letters and symbols
    df = df.replace(r"[^\d./]+","", regex=True)
    #remove the spaces
    df = df.replace(r" ", "", regex=True)
    #expand the price and weight
    split_df = df["est_price"].str.split("/", expand=True)
    df[currency] = split_df[0]
    df[weight] = split_df[1]
    #drop the est_price column
    df.drop("est_price", axis=1, inplace=True)
    return df

In [53]:
CAD_prices = clean_df(CAD_prices, "CAD", "weight")
CAD_prices[:10]

,CAD,weight
9,17.00,12
12,17.00,16
29,18.00,12
30,18.00,12
31,21.00,12
128,15.50,340
163,29.00,200
212,16.00,16
217,17.00,12
231,15.00,250


In [54]:
#There were two different measurements, ounces and grams. We can tell the difference by the number of digits
CAD_prices["grams"] = CAD_prices["weight"].str.extract("([\w]{3})")
CAD_prices

,CAD,weight,grams
9,17.00,12,NaN
12,17.00,16,NaN
29,18.00,12,NaN
30,18.00,12,NaN
31,21.00,12,NaN
128,15.50,340,340
163,29.00,200,200
212,16.00,16,NaN
217,17.00,12,NaN
231,15.00,250,250


In [55]:
#Convert ounces to grams

CAD_prices["ounces_to_grams"] = CAD_prices["weight"].replace(r"[\w]{3}", 0, regex=True)
CAD_prices["ounces_to_grams"] = CAD_prices["ounces_to_grams"].astype("int") * 28.3
CAD_prices["ounces_to_grams"] = CAD_prices["ounces_to_grams"].replace(0, np.nan, regex=True)
CAD_prices.drop("weight", axis=1, inplace=True)
CAD_prices

,CAD,grams,ounces_to_grams
9,17.00,NaN,339.6
12,17.00,NaN,452.8
29,18.00,NaN,339.6
30,18.00,NaN,339.6
31,21.00,NaN,339.6
128,15.50,340,NaN
163,29.00,200,NaN
212,16.00,NaN,452.8
217,17.00,NaN,339.6
231,15.00,250,NaN


In [56]:
CAD_prices["grams"] = CAD_prices["grams"].fillna(CAD_prices["ounces_to_grams"])
CAD_prices["grams"]
CAD_prices.drop("ounces_to_grams", axis=1, inplace=True)
CAD_prices

,CAD,grams
9,17.00,339.6
12,17.00,452.8
29,18.00,339.6
30,18.00,339.6
31,21.00,339.6
128,15.50,340
163,29.00,200
212,16.00,452.8
217,17.00,339.6
231,15.00,250


In [57]:
#Finally convert to USD
CAD_prices["USD_per_gram"] = CAD_prices["CAD"].astype("float").astype("int") / CAD_prices["grams"].astype("float").astype("int") * 0.75
CAD_prices["100g_USD"] = round(CAD_prices["USD_per_gram"] * 100, 2)
CAD_prices.drop(["CAD", "grams", "USD_per_gram"], axis=1, inplace=True)
CAD_prices

,100g_USD
9,3.76
12,2.82
29,3.98
30,3.98
31,4.65
128,3.31
163,10.87
212,2.65
217,3.76
231,4.50


In [58]:
NT_prices = df.loc[df["NT"].notna() == True, ["est_price"]]
NT_prices[:10]

,est_price
11,NT $550/16 ounces
26,NT $650/16 ounces
28,NT $550/16 ounces
52,NT $580/220 grams
63,NT $2500/16 ounces
78,NT $500/16 ounces
80,NT $500/16 ounces
100,NT $1300/16 ounces
106,NT $550/16 ounces
107,NT $600/16 ounces


In [59]:
NT_prices = clean_df(NT_prices, "NT", "weight")
NT_prices

,NT,weight
11,550,16
26,650,16
28,550,16
52,580,220
63,2500,16
...,...,...
2256,500,4
2259,800,20
2267,550,200
2268,380,200


In [60]:
# separate grams from ounces
NT_prices["grams"] = NT_prices["weight"].str.extract("([\w]{3})")
# replace 3-digit values with 0 in the ounces column
NT_prices["ounces_to_grams"] = NT_prices["weight"].replace(r"[\w]{3}", 0, regex=True)

In [61]:
def ounces_to_grams(df, column):
    # convert ounces to grams by multiplying it by 28.3
    df["ounces_to_grams"] = df[column].astype("float") * 28.3
    # turn all 0 values to np.nan
    df["ounces_to_grams"] = df["ounces_to_grams"].replace(0, np.nan, regex=True)
    # fill grams column with values from our converted column
    df["grams"] = df["grams"].fillna(df["ounces_to_grams"])
    # drop the unnecessary columns
    df.drop(["ounces_to_grams"], axis=1, inplace=True)
    return df

In [62]:
NT_prices = ounces_to_grams(NT_prices, "ounces_to_grams")
NT_prices

,NT,weight,grams
11,550,16,452.8
26,650,16,452.8
28,550,16,452.8
52,580,220,220
63,2500,16,452.8
...,...,...,...
2256,500,4,113.2
2259,800,20,566.0
2267,550,200,200
2268,380,200,200


In [63]:
#Should have done this earlier but lets make a function to easy convert these
def convert_currency(df, currency, exchange_rate):
    df["USD_per_gram"] = df[currency].astype("float").astype("int") / df["grams"].astype("float").astype("int") * exchange_rate
    df["100g_USD"] = round(df["USD_per_gram"] * 100, 2)
    df.drop([currency, "grams", "USD_per_gram"], axis=1, inplace=True)
    return df

In [64]:
NT_prices = convert_currency(NT_prices, "NT", 0.032)
NT_prices = NT_prices.drop("weight", axis=1)
NT_prices

,100g_USD
11,3.89
26,4.60
28,3.89
52,8.44
63,17.70
...,...
2256,14.16
2259,4.52
2267,8.80
2268,6.08


In [65]:
JPY_prices = df.loc[df["JPY"].notna() == True, ["est_price"]]
JPY_prices

,est_price
1204,"¥ 2,420/200 grams"
1205,"¥ 2,420/200 grams"
1309,"¥ 1,580/100 grams"
1310,"¥ 2,640/50 grams"
1410,¥1280/100 grams
1411,¥1280/100 grams
1412,¥1280/100 grams
1413,¥1280/100 grams
1474,"¥1,050/100 grams"
1505,¥2980/100 grams


In [66]:
#The only 3 that use ounces are a drastic change in cost. This must be a typo so we will drop them.

JPY_prices = JPY_prices.drop([1693, 1696])

In [67]:
#Combine both functions into one.

def clean_and_exchange(df, currency, weight, exchange_rate):
    cleaned_df = clean_df(df, currency, weight)
    converted_df = convert_currency(cleaned_df, currency, exchange_rate)
    return converted_df

In [68]:
JPY_prices = clean_and_exchange(JPY_prices, "JPY", "grams", 0.0072)
JPY_prices

,100g_USD
1204,8.71
1205,8.71
1309,11.38
1310,38.02
1410,9.22
1411,9.22
1412,9.22
1413,9.22
1474,7.56
1505,21.46


In [69]:
EUR_prices = df.loc[df["EUR"].notna() == True, ["est_price"]]
EUR_prices

,est_price
789,€29.95/1 kilo (35.3 ounces)


In [70]:
clean_df(EUR_prices, "EUR", "grams")

,EUR,grams
789,29.95,135.3


In [71]:
#Since this had an odd weight, we are going to have to do something different

# remove the letters and special symbols
EUR_prices = EUR_prices.replace(r"[A-Za-z$¥£€#*;(),-]","", regex=True)
# separate price from weight
split_df = EUR_prices["est_price"].str.split("/", expand=True)
# use the price column only 
EUR_prices["EUR"] = split_df[0]
# find the cost of 1g of coffee in USD by dividing it by our weight (1000g) and multiplying it by the exchange rate
EUR_prices["USD_per_gram"] = EUR_prices["EUR"].astype("float").astype("int") / 1000 * 1.04
# multiply the result by 100 and round it up to get price for 100g
EUR_prices["100g_USD"] = round(EUR_prices["USD_per_gram"] * 100, 2)
# drop the unnecessary columns
EUR_prices.drop(["est_price", "EUR", "USD_per_gram"], axis=1, inplace=True)

In [72]:
EUR_prices

,100g_USD
789,3.02


In [73]:
GBP_prices = df.loc[df["GBP"].notna() == True, ["est_price"]]
GBP_prices

,est_price
270,£50/10 capsules
271,£45/10 capsules
613,£40.50/10 capsules
921,£50.00/10 capsules
1129,£45/10 capsules
1526,£50/10 capsules
1650,£25.00/8 18-gram pouches
1776,£100/150-gram packet
2074,£50/10 capsules


In [74]:
#10 cpasules is weird. Let's try to find more info
GBP_prices["name"] = df.loc[df["GBP"].notna() == True, ["name"]]
GBP_prices

,est_price,name
270,£50/10 capsules,Esmeralda Estate Panama Geisha
271,£45/10 capsules,Brazil Cup of Excellence Prize Winner
613,£40.50/10 capsules,Cup of Excellence Rwanda Champion
921,£50.00/10 capsules,Esmeralda Estate Panama Geisha
1129,£45/10 capsules,Esmeralda Estate Panama Geisha
1526,£50/10 capsules,Esmeralda Estate Panama Geisha
1650,£25.00/8 18-gram pouches,Bait Alal Community
1776,£100/150-gram packet,Hacienda La Esmeralda Montañas 71AS
2074,£50/10 capsules,Esmeralda Estate Panama Geisha


In [75]:
#After looking at the names, I found that Esmeralda has 5.4 gram per capsule. 
#Couldn't find info on the other brands so we can just drop those

df = df.drop(index=[271, 613])
GBP_prices = GBP_prices.drop(index=[271,613])
GBP_prices

,est_price,name
270,£50/10 capsules,Esmeralda Estate Panama Geisha
921,£50.00/10 capsules,Esmeralda Estate Panama Geisha
1129,£45/10 capsules,Esmeralda Estate Panama Geisha
1526,£50/10 capsules,Esmeralda Estate Panama Geisha
1650,£25.00/8 18-gram pouches,Bait Alal Community
1776,£100/150-gram packet,Hacienda La Esmeralda Montañas 71AS
2074,£50/10 capsules,Esmeralda Estate Panama Geisha


In [76]:
GBP_prices = clean_df(GBP_prices, "GBP", "grams")
GBP_prices.drop("name", axis = 1, inplace= True)
GBP_prices

,GBP,grams
270,50,10
921,50.00,10
1129,45,10
1526,50,10
1650,25.00,818
1776,100,150
2074,50,10


In [77]:
GBP_prices.loc[GBP_prices["grams"] == "10", "grams"] = 54
GBP_prices.loc[GBP_prices["grams"] == "818", "grams"] = 144

GBP_prices

,GBP,grams
270,50,54
921,50.00,54
1129,45,54
1526,50,54
1650,25.00,144
1776,100,150
2074,50,54


In [78]:
GBP_prices = convert_currency(GBP_prices, "GBP", 1.2)

In [79]:
GBP_prices

,100g_USD
270,111.11
921,111.11
1129,100.00
1526,111.11
1650,20.83
1776,80.00
2074,111.11


In [80]:
#Let's combine all the data frames now.

price_df = pd.concat([JPY_prices, CAD_prices, NT_prices, HKD_prices, GBP_prices, EUR_prices]).sort_index(axis=0)
price_df

,100g_USD
0,14.32
1,9.05
9,3.76
11,3.89
12,2.82
...,...
2256,14.16
2259,4.52
2267,8.80
2268,6.08


In [81]:
#Let's now create a column for price and ounces

ounces = df.loc[df["ounces"].notnull() & (df["CAD"].isnull()) & (df["NT"].isnull()) & (df["EUR"].isnull())]

ounces

,rating,roaster,name,roast,est_price,review_date,desc_1,desc_2,desc_3,loc_country,origin_1,origin_2,grams,ounces,HKD,CAD,NT,JPY,EUR,GBP
2,92,Revel Coffee,Ethiopia Shakiso Mormora,Medium-Light,$16.00/12 ounces,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...",United States,Guji Zone,Southern Ethiopia,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
3,92,Roast House,Ethiopia Suke Quto,Medium-Light,$19.00/16 ounces,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,United States,Guji Zone,Oromia Region,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
4,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,Medium,$16.50/12 ounces,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,United States,Gedeb District,Gedeo Zone,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
5,93,Red Rooster Coffee Roaster,Ethiopia Kayon Mountain,Light,$17.49/12 ounces,November 2017,"Delicate, richly and sweetly tart. Dried hibis...",This coffee tied for the second-highest rating...,"A lively and crisply sweet, fruit-forward natu...",United States,Odo Shakiso District,Guji Zone,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
6,93,Willoughby's Coffee & Tea,Ethiopia Gelgelu Natural Organic,Medium-Light,$17.99/16 ounces,November 2017,"High-toned, floral. Dried apricot, magnolia, a...",This coffee tied for the second-highest rating...,"A deeply sweet natural-processed Ethiopia cup,...",United States,Yirgacheffe Growing Region,Southern Ethiopia,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,92,El Gran Cafe,Finca Patzibir,Medium-Light,$16.00/12 ounces,November 2022,"Crisply sweet, nut-toned. Almond brittle, pie ...",Produced by Andres Fahsen of Finca Patzibar en...,"A quietly confident, sweetly nut-toned Guatema...",Guatemala,Lake Atitlán Growing Region,Guatemala,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
2278,93,El Gran Cafe,Proyecto Aurora,Medium-Light,$10.00/12 ounces,November 2022,"Chocolaty, floral-framed. Cocoa nib, honeysuck...",Produced at Beneficio La Esperanza from trees ...,"A deeply floral, richly chocolaty Guatemala cu...",Guatemala,Antigua Growing Region,Guatemala,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
2279,93,El Gran Cafe,Finca El Potrero,Medium-Light,$10.00/12 ounces,November 2022,"High-toned, enticingly sweet. Black cherry, na...",Produced by Josué Morales of Finca El Potrero...,"A bright, balanced, juicy Guatemala cup driven...",Guatemala,Antigua Growing Region,Guatemala,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN
2280,93,El Gran Cafe,Chacayá Santiago Atitlán,Medium-Light,$16.00/12 ounces,November 2022,"Vibrantly sweet, subtly nuanced. Apricot, dar...",Produced by smallholding farmers of Asociación...,"Balanced, bright, invigoratingly crisp, with t...",Guatemala,Lake Atitlán Growing Region,Guatemala,NaN,ounces,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
ounce_prices = ounces.loc[:, ["est_price"]]
ounce_prices

,est_price
2,$16.00/12 ounces
3,$19.00/16 ounces
4,$16.50/12 ounces
5,$17.49/12 ounces
6,$17.99/16 ounces
...,...
2277,$16.00/12 ounces
2278,$10.00/12 ounces
2279,$10.00/12 ounces
2280,$16.00/12 ounces


In [83]:
# pattern: $ at the beginning of the string, a group of one or more digits with a dot (".") to indicate cents, slash, one or more digits, and the word "ounces"
pattern = r'^\$(\d+\.\d+)/(\d+)\sounces$'

#create a function that returns a boolean if the pattern is matched
def find_match(x):
    return bool(re.findall(pattern, x))

# create a new column with the output of find_match function
ounce_prices["match"] = ounce_prices["est_price"].apply(find_match)

In [84]:
# show only the rows that don't match our pattern
ounce_prices.loc[ounce_prices["match"]==False]

,est_price,match
218,$19.50/12 ounces online; $12.99/ounces in stores,False
241,$84.00/24 12-ounces bottles,False
294,$8.99/8 ounces (226 grams),False
296,$9.99/7 ounces (198 grams),False
399,#23.00/12 ounces,False
547,18.00/12 ounces,False
651,$39.95/8 ounces*,False
675,"$39.95/8 ounces (packaged as a ""duo"" with Bour...",False
676,"$39.95/8 ounces (packaged as a ""duo"" with the ...",False
697,$125.00/4 ounces; limited availability,False


In [85]:
to_remove = [218, 697, 802, 842, 846, 849, 934, 936, 1693, 1696]

ounce_prices.drop(to_remove, axis=0, inplace=True)

In [86]:
pattern = r'^[A-Z]{2,3}'

def find_match(x):
    return bool(re.findall(pattern, x))

ounce_prices["match"] = ounce_prices["est_price"].apply(find_match)
ounce_prices.loc[ounce_prices["match"]==True]

,est_price,match
1121,US $20.00/12 ounces,True
1190,USD $206.00/8 ounces,True
1191,USD $206.00/8 ounces,True


In [87]:
ounce_prices.drop("match", axis=1, inplace=True)

In [88]:
ounce_prices = clean_df(ounce_prices, "USD", "ounces")

In [89]:
def ounces_to_100g(df):
    df["grams"] = df["ounces"].astype("float64") * 28.35
    df["price_per_gram"] = df["USD"].astype("float64") / df["grams"].astype("float64")
    df["100g_USD"] = round(df["price_per_gram"] * 100, 2)
    df.drop(["grams", "ounces", "price_per_gram", "USD"], axis=1, inplace=True)
    return df

In [90]:
ounce_prices = ounces_to_100g(ounce_prices)
ounce_prices

,100g_USD
2,4.70
3,4.19
4,4.85
5,5.14
6,3.97
...,...
2277,4.70
2278,2.94
2279,2.94
2280,4.70


In [91]:
price_df

,100g_USD
0,14.32
1,9.05
9,3.76
11,3.89
12,2.82
...,...
2256,14.16
2259,4.52
2267,8.80
2268,6.08


In [92]:
print(f"Ounce_prices df mean: {ounce_prices.mean()}")
print(f"Price_df mean: {price_df.mean()}")

Ounce_prices df mean: 100g_USD    9.111869
dtype: float64
Price_df mean: 100g_USD    9.457388
dtype: float64


In [93]:
#Check for outlier countries
df["loc_country"].unique()

array(['Hong Kong', 'United States', 'Canada', 'Taiwan', "Hawai'i",
       'Australia', 'Thailand', 'South Korea', 'England', 'China',
       'Japan', 'United Arab Emirates', 'Uganda', 'Slovakia', 'Mexico',
       'Indonesia', 'Laos', 'Belgium', 'United States And Floyd',
       'Guatemala', 'Peru', 'Honduras', 'Kenya', 'Malaysia', 'New Taiwan'],
      dtype=object)

In [94]:
countries = ["China", "Australia", "Thailand", "South Korea", "United Arab Emirates", "Uganda", "Slovakia", "Mexico", "Indonesia", "Laos", "Belgium", "Guatemala", "Peru", "Honduras", "Colombia", "Kenya", "Malaysia"]

df.loc[df["loc_country"].isin(countries), ["est_price","loc_country"]]

,est_price,loc_country
61,AUD $48.00/200 grams,Australia
108,$6.00/250 grams,Thailand
133,"KRW $11,000/250 grams",South Korea
170,THB $320/250 grams,Thailand
274,"KRW $14,000/200 grams",South Korea
...,...,...
2277,$16.00/12 ounces,Guatemala
2278,$10.00/12 ounces,Guatemala
2279,$10.00/12 ounces,Guatemala
2280,$16.00/12 ounces,Guatemala


In [95]:
#Let's fix the AUD
AUD_prices = df.loc[[61,678], ["est_price"]]
AUD_prices

,est_price
61,AUD $48.00/200 grams
678,AUD $18.00/18 grams


In [96]:
AUD_prices = clean_and_exchange(AUD_prices, "AUD", "grams", 0.69)
AUD_prices

,100g_USD
61,16.56
678,69.00


In [97]:
#Now Peru
Peru_prices = df.loc[[1206,1207], ["est_price"]]
Peru_prices

,est_price
1206,$10.00/250 grams
1207,$10.00/250 grams


In [98]:
Peru_prices = clean_and_exchange(Peru_prices, "USD", "grams", 1)
Peru_prices

,100g_USD
1206,4.0
1207,4.0


In [99]:
#Now Malaysia
RM_prices = df.loc[[2069,2070,2071], ["est_price"]]
RM_prices

,est_price
2069,RM 130/350 grams
2070,RM 120/350 grams
2071,RM 120/350 grams


In [100]:
RM_prices = clean_and_exchange(RM_prices, "RM", "grams", 0.23)
RM_prices

,100g_USD
2069,8.54
2070,7.89
2071,7.89


In [101]:
additional_prices = pd.concat([AUD_prices, Peru_prices, RM_prices]).sort_index(axis=0)

In [102]:
all_prices = pd.concat([price_df, ounce_prices, additional_prices]).sort_index(axis=0)
all_prices

,100g_USD
0,14.32
1,9.05
2,4.70
3,4.19
4,4.85
...,...
2277,4.70
2278,2.94
2279,2.94
2280,4.70


In [103]:
clean_df = df.merge(all_prices, left_index=True, right_index=True)
clean_df = clean_df.drop(["est_price", "grams", "ounces", "HKD", "CAD", "NT", "JPY", "EUR", "GBP"], axis=1)
clean_df[:15]

,rating,roaster,name,roast,review_date,desc_1,desc_2,desc_3,loc_country,origin_1,origin_2,100g_USD
0,95,A.R.C.,“Sweety” Espresso Blend,Medium-Light,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,Hong Kong,Panama,Ethiopia,14.32
1,94,A.R.C.,Flora Blend Espresso,Medium-Light,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...",Hong Kong,Africa,Asia Pacific,9.05
2,92,Revel Coffee,Ethiopia Shakiso Mormora,Medium-Light,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...",United States,Guji Zone,Southern Ethiopia,4.70
3,92,Roast House,Ethiopia Suke Quto,Medium-Light,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,United States,Guji Zone,Oromia Region,4.19
4,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,Medium,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,United States,Gedeb District,Gedeo Zone,4.85
5,93,Red Rooster Coffee Roaster,Ethiopia Kayon Mountain,Light,November 2017,"Delicate, richly and sweetly tart. Dried hibis...",This coffee tied for the second-highest rating...,"A lively and crisply sweet, fruit-forward natu...",United States,Odo Shakiso District,Guji Zone,5.14
6,93,Willoughby's Coffee & Tea,Ethiopia Gelgelu Natural Organic,Medium-Light,November 2017,"High-toned, floral. Dried apricot, magnolia, a...",This coffee tied for the second-highest rating...,"A deeply sweet natural-processed Ethiopia cup,...",United States,Yirgacheffe Growing Region,Southern Ethiopia,3.97
7,93,Black Oak Coffee Roasters,Ethiopia Hambela Alaka,Medium-Light,November 2017,"Very delicate, sweetly savory. Lemon verbena, ...",This coffee tied for the second-highest rating...,"A delicate, richly sweet wet-processed Ethiopi...",United States,Hambela Wamena District,Guji Zone,5.14
8,93,Wonderstate Coffee,Organic Ethiopia Kirite,Medium-Light,November 2017,"High-toned, crisply sweet. Lemon blossom, apri...",This coffee tied for the second-highest rating...,An inviting wet-processed Ethiopia cup. Invoke...,United States,Hambela District,Guji Zone,5.29
9,94,Reunion Island Coffee,Ethiopia Sidama,Medium,November 2017,"Balanced, sweet-savory. Red currant, roasted c...",This exceptional coffee was selected as the No...,"An elegantly expressive, fruit-forward but sav...",Canada,Sidama (Also Sidamo) Growing Region,Southern Ethiopia,3.76


In [104]:
#Organizing the columns
clean_df = clean_df[["name", "roaster", "roast", "loc_country", "origin_1", "origin_2", "100g_USD", "rating", "review_date", "desc_1", "desc_2", "desc_3"]]

In [105]:
clean_df.head()

,name,roaster,roast,loc_country,origin_1,origin_2,100g_USD,rating,review_date,desc_1,desc_2,desc_3
0,“Sweety” Espresso Blend,A.R.C.,Medium-Light,Hong Kong,Panama,Ethiopia,14.32,95,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,Flora Blend Espresso,A.R.C.,Medium-Light,Hong Kong,Africa,Asia Pacific,9.05,94,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,Ethiopia Shakiso Mormora,Revel Coffee,Medium-Light,United States,Guji Zone,Southern Ethiopia,4.70,92,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,Ethiopia Suke Quto,Roast House,Medium-Light,United States,Guji Zone,Oromia Region,4.19,92,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,Ethiopia Gedeb Halo Beriti,Big Creek Coffee Roasters,Medium,United States,Gedeb District,Gedeo Zone,4.85,94,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [106]:
clean_df.to_csv("coffee_analysis_clean.csv", index=False)